In [1]:
!pip install -q pytest pytest-cov

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.5/251.5 kB 15.4 MB/s eta 0:00:00


In [3]:
from huggingface_hub import login

# Paste your token when prompted
login()

In [4]:
from datasets import load_dataset

dataset = load_dataset("openai/openai_humaneval", split="test")
print(dataset.column_names)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

openai_humaneval/test-00000-of-00001.par(…):   0%|          | 0.00/83.9k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point']


In [5]:
from datasets import load_dataset
import pandas as pd

n_problems = 10  # choose how many problems you want to load

print("Loading dataset...")

dataset = load_dataset("openai/openai_humaneval", split="test")[:n_problems]

# Convert to DataFrame
df_tests = pd.DataFrame({
    'task_id': dataset['task_id'],
    'prompt': dataset['prompt'],
    'canonical_solution': dataset['canonical_solution'],
    'test': dataset['test'],
    'entry_point': dataset['entry_point']
})

print(df_tests.head())


Loading dataset...
       task_id                                             prompt  \
0  HumanEval/0  from typing import List\n\n\ndef has_close_ele...   
1  HumanEval/1  from typing import List\n\n\ndef separate_pare...   
2  HumanEval/2  \n\ndef truncate_number(number: float) -> floa...   
3  HumanEval/3  from typing import List\n\n\ndef below_zero(op...   
4  HumanEval/4  from typing import List\n\n\ndef mean_absolute...   

                                  canonical_solution  \
0      for idx, elem in enumerate(numbers):\n    ...   
1      result = []\n    current_string = []\n    ...   
2                              return number % 1.0\n   
3      balance = 0\n\n    for op in operations:\n...   
4      mean = sum(numbers) / len(numbers)\n    re...   

                                                test              entry_point  
0  \n\nMETADATA = {\n    'author': 'jt',\n    'da...       has_close_elements  
1  \n\nMETADATA = {\n    'author': 'jt',\n    'da...    separate_pare

---

In [9]:
try:
    # Load your CSVs
    df_code = pd.read_csv('/content/part1_results.csv', usecols=['generated_code'])
    print(f"Loaded {len(df_code)} code snippets.")
    print(df_code.head)
except FileNotFoundError:
    print("ERROR: Could not find .csv.")
    print("Please make sure your files are in the correct location.")
except ValueError as e:
    print(f"ERROR: A column might be missing. Make sure you have a 'code' and 'tests' column.")
    print(f"Details: {e}")

Loaded 120 code snippets.
<bound method NDFrame.head of                                         generated_code
0    from typing import List\n\n\ndef has_close_ele...
1    from typing import List\n\ndef has_close_eleme...
2    Below is a clean, production‑ready implementat...
3    print(has_close_elements([1.0, 2.0, 3.0], 0.5)...
4    def has_close_elements(numbers: List[float], t...
..                                                 ...
115  def is_prime(num: int) -> bool:\n    """\n    ...
116  def _is_prime(n: int) -> bool:\n    """\n    D...
117  def is_prime(num: int) -> bool:\n    """\n    ...
118  def _is_prime(n: int) -> bool:\n    """\n    R...
119  def is_prime(num: int) -> bool:\n    """\n    ...

[120 rows x 1 columns]>


In [10]:
import os
import math
import pandas as pd

def _decode_text(s):
    """Decode escaped sequences if s is a string, else return as-is."""
    if not isinstance(s, str):
        return s
    try:
        return s.encode("utf-8").decode("unicode_escape")
    except Exception:
        return s

def _normalize_tests_for_injection(tests_cell):
    """
    Return a list of test lines (strings) to insert inside the test function.
    Handles:
      - actual Python lists of assertions
      - string like "[assert ... , assert ...]"
      - multiline strings with '\n' escapes
    """
    if isinstance(tests_cell, list):
        # assume already list of assertion strings
        return [ _decode_text(str(x)).strip() for x in tests_cell if str(x).strip() ]

    if not isinstance(tests_cell, str):
        return []

    s = _decode_text(tests_cell).strip()

    # strip surrounding brackets if present
    if s.startswith("[") and s.endswith("]"):
        s = s[1:-1].strip()

    # if multiple assertions separated by ', assert' -> split nicely
    # also handle actual newlines
    if "\n" in s:
        lines = [line.strip() for line in s.splitlines() if line.strip()]
    elif ", assert" in s:
        # keep the leading "assert" on the first element
        parts = s.split(", assert")
        lines = []
        for idx, p in enumerate(parts):
            p = p.strip()
            if idx > 0 and not p.startswith("assert"):
                p = "assert " + p
            lines.append(p)
    elif "assert" in s:
        # single-line with one assert (or semicolon-separated)
        if ";" in s:
            parts = [p.strip() for p in s.split(";") if p.strip()]
            lines = [p if p.startswith("assert") else ("assert " + p) for p in parts]
        else:
            lines = [s]
    else:
        # no 'assert' found; treat whole thing as single line
        lines = [s] if s else []

    # final clean-up: ensure each line is a valid expression string
    return [line for line in lines if line]

def generate_test_files(df_code, df_tests, output_dir="generated_tests"):
    """
    For each row in df_tests (index t_idx), create up to 5 files corresponding to
    code rows at indices (t_idx*5 + iter_idx) where iter_idx in 0..4.
    Each file contains the code snippet (decoded from escaped form) followed by
    a test function `test_generated_snippet()` containing the test assertions.
    Files are named: problem_{t_idx}_iteration_{iter_idx}.py
    """
    os.makedirs(output_dir, exist_ok=True)

    n_code = len(df_code)
    n_tests = len(df_tests)

    for t_idx, tests_row in df_tests.iterrows():
        raw_tests = tests_row.get("tests", "")
        test_lines = _normalize_tests_for_injection(raw_tests)

        for iter_idx in range(5):
            code_idx = t_idx * 5 + iter_idx
            if code_idx >= n_code:
                # no corresponding code row — skip
                continue

            code_cell = df_code.iloc[code_idx].get("code", "")
            if not isinstance(code_cell, str) or not code_cell.strip():
                # skip empty code cells
                continue

            # decode escaped newlines/tabs etc.
            code_text = _decode_text(code_cell).rstrip()

            # ensure code ends with exactly one newline
            if not code_text.endswith("\n"):
                code_text = code_text + "\n"

            # build the test function body
            file_lines = []
            file_lines.append(code_text)
            file_lines.append("def test_generated_snippet():\n")

            if test_lines:
                for line in test_lines:
                    # indent each test line with 4 spaces
                    file_lines.append(f"    {line.rstrip()}\n")
            else:
                file_lines.append("    pass\n")

            # final content
            file_content = "".join(file_lines)

            # write file
            filename = os.path.join(output_dir, f"problem_{t_idx}_iteration_{iter_idx}.py")
            with open(filename, "w", encoding="utf-8") as f:
                f.write(file_content)

    print(f"✅ Generated test files in '{output_dir}' (tests outer loop, code inner loop).")

generate_test_files(df_code, df_tests)


# Example usage:
# generate_test_files(df_code, df_tests)


✅ Generated test files in 'generated_tests' (tests outer loop, code inner loop).


In [12]:
import os
import subprocess
import sys
from pathlib import Path

# --- Configuration ---
# V V V V V V V V V V V V V V V V V V V V V V V V V V V
# --- EDIT THESE VARIABLES ---
#
# Point to your two new folders
test_directory = "/content/generated_tests"
csv_file_path = "/content/part1_results.csv"
code_directory = "./generated_code"
#
# ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^
# --- End Configuration ---



def prepare_code_from_csv(csv_path: str, output_dir: str):
    """
    Reads the CSV file and saves each code snippet from the 'generated_code' column
    as a separate Python file in the output directory.
    """
    df = pd.read_csv(csv_path)
    if "generated_code" not in df.columns:
        raise ValueError("CSV file must contain a column named 'generated_code'")

    os.makedirs(output_dir, exist_ok=True)

    for i, code in enumerate(df["generated_code"], start=1):
        file_path = Path(output_dir) / f"code_{i}.py"
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(str(code))
    print(f"✅ Extracted {len(df)} code files into '{output_dir}'")


def run_all_tests(test_dir_str: str, src_dir_str: str):
    """
    Runs pytest-cov on all tests and generates coverage reports.
    """
    test_dir = Path(test_dir_str)
    src_dir = Path(src_dir_str)

    if not test_dir.is_dir():
        print(f"❌ Error: Test folder not found: {test_dir.resolve()}")
        return
    if not src_dir.is_dir():
        print(f"❌ Error: Code folder not found: {src_dir.resolve()}")
        return

    print("🚀 Starting test run...")
    print(f"Running tests in: {test_dir.resolve()}")
    print(f"Measuring coverage for: {src_dir.resolve()}")
    print("=" * 70)

    # Clear old coverage data
    subprocess.run("coverage erase", shell=True, capture_output=True)

    # Run pytest with coverage
    command = [
        sys.executable,
        "-m", "pytest",
        f"--cov={src_dir}",
        "-v",
        "--cov-branch",
        test_dir
    ]

    subprocess.run(command, text=True)

    print("=" * 70)
    print("✅ All tests finished.")
    print("📊 Generating final combined coverage report...")
    print("=" * 70)

    subprocess.run([sys.executable, "-m", "coverage", "report", "-m"])
    subprocess.run([sys.executable, "-m", "coverage", "html"])

    report_path = Path.cwd() / 'htmlcov' / 'index.html'
    print(f"\n🌐 Full HTML report generated: {report_path}")
    print("=" * 70)
    print("🎉 Script finished.")


# --- MAIN EXECUTION ---

# Step 1: Prepare generated_code/ folder from CSV
prepare_code_from_csv(csv_file_path, code_directory)

# Step 2: Run all tests
run_all_tests(test_directory, code_directory)


✅ Extracted 120 code files into './generated_code'
🚀 Starting test run...
Running tests in: /content/generated_tests
Measuring coverage for: /content/generated_code
✅ All tests finished.
📊 Generating final combined coverage report...

🌐 Full HTML report generated: /content/htmlcov/index.html
🎉 Script finished.


In [13]:
import os
import ast
import sys
from pathlib import Path

# --- Configuration ---
# V V V V V V V V V V V V V V V V V V V V V V V V V V V
# --- 1. SET YOUR FOLDER PATH ---
#
# Point this to the folder containing your 'problem_*.py' files.
target_directory = "./generated_code"  # <--- SET THIS

# --- 2. SET 'DRY_RUN' ---
#
# True = Print what changes WOULD be made, without touching any files.
# False = Actually modify and save the files.
#
# !! ALWAYS RUN WITH 'True' FIRST TO CHECK THE OUTPUT !!
#
DRY_RUN = False
#
# ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^
# --- End Configuration ---


class TestFunctionRemover(ast.NodeTransformer):
    """
    This class visits every node in a Python file's AST.
    If it finds a function definition (FunctionDef) whose name
    starts with 'test_', it returns 'None', which effectively
    removes it from the tree.
    """
    def __init__(self):
        self.functions_removed_count = 0
        super().__init__()

    def visit_FunctionDef(self, node):
        if node.name.startswith('test_'):
            self.functions_removed_count += 1
            return None  # Remove this node
        return node  # Keep this node


def clean_test_functions(target_dir_str: str, dry_run: bool):
    """
    Main function to process files.
    """

    # --- !! SAFETY WARNING !! ---
    if not dry_run:
        print("=" * 70)
        print("!! WARNING: LIVE RUN !!")
        print("This script is about to permanently modify your files.")
        print("PLEASE MAKE A BACKUP of your folder first.")
        print(f"Target folder: {target_dir_str}")
        print("You have 5 seconds to cancel (Ctrl+C)...")
        print("=" * 70)
        try:
            import time
            time.sleep(5)
        except KeyboardInterrupt:
            print("\nOperation cancelled by user.")
            return
    else:
        print("=" * 70)
        print("DRY RUN MODE - No files will be modified.")
        print("=" * 70)
    # --- End Warning ---

    target_dir = Path(target_dir_str)

    if not target_dir.is_dir():
        print(f"Error: Folder not found: {target_dir.resolve()}")
        return

    files_processed = 0
    files_to_modify = 0
    total_funcs_removed = 0

    # Use rglob to find 'problem_*.py' files in all subdirectories
    for file_path in target_dir.rglob('problem_*.py'):
        files_processed += 1
        print(f"\n--- Processing: {file_path.relative_to(target_dir)} ---")

        try:
            # Read the original file content
            content = file_path.read_text()
            if not content.strip():
                print("File is empty. Skipping.")
                continue

            # Parse the content into an Abstract Syntax Tree
            tree = ast.parse(content)

            # Create our remover and have it "visit" the tree
            remover = TestFunctionRemover()
            new_tree = remover.visit(tree)

            if remover.functions_removed_count > 0:
                files_to_modify += 1
                total_funcs_removed += remover.functions_removed_count

                # Turn the modified AST back into Python source code
                # Note: ast.unparse() adds newlines at the end
                new_content = ast.unparse(new_tree)

                if dry_run:
                    print(f"DRY RUN: Would remove {remover.functions_removed_count} 'test_' function(s).")
                else:
                    # Write the modified content back to the file
                    file_path.write_text(new_content)
                    print(f"SUCCESS: Removed {remover.functions_removed_count} 'test_' function(s) and saved file.")
            else:
                print("No 'test_' functions found. Skipping.")

        except SyntaxError as e:
            print(f"SYNTAX ERROR: Could not parse file. Skipping.")
            print(f"  Error details: {e}")
        except Exception as e:
            print(f"UNEXPECTED ERROR during processing: {e}. Skipping file.")

    # --- Final Summary ---
    print("\n" + "=" * 70)
    print("Script Finished.")
    print(f"Total files scanned: {files_processed}")
    if dry_run:
        print(f"Files that WOULD be modified: {files_to_modify}")
        print(f"Total 'test_' functions that WOULD be removed: {total_funcs_removed}")
        print("\nTo apply these changes, set DRY_RUN = False and re-run the cell.")
    else:
        print(f"Files ACTUALLY modified: {files_to_modify}")
        print(f"Total 'test_' functions ACTUALLY removed: {total_funcs_removed}")
        print("Operation complete.")
    print("=" * 70)

# --- Run the function ---
clean_test_functions(target_directory, DRY_RUN)

!! WARNING: LIVE RUN !!
This script is about to permanently modify your files.
PLEASE MAKE A BACKUP of your folder first.
Target folder: ./generated_code
You have 5 seconds to cancel (Ctrl+C)...

Script Finished.
Total files scanned: 0
Files ACTUALLY modified: 0
Total 'test_' functions ACTUALLY removed: 0
Operation complete.
